In [ ]:
from enum import Enum

import pandas as pd

from IPython.display import display

# Regression

In [ ]:
class OutputFiles:
    regression_no_average = "regression-overview.csv"
    regession_set_size_average = "regression-overview-averaged.csv"

class TaskType(Enum):
    REGRESSION = 0
    CLASSIFICATION = 1

class ModelType(Enum):
    DEEPSETS_MLP_SUM = "deepsets_mlp_sum"
    DEEPSETS_MLP_FSPOOL = "deepsets_mlp_fspool"
    # MLP = "mlp"
    SORTED_MLP = "sorted_mlp"
    SMALL_SET_TRANSFORMER = "small_set_transformer"
    PNA = "pna"

class Task(Enum):
    SUM = "sum"
    LARGEST_PAIR_SUM = "largest_pair_sum"
    LARGEST_TRIPLE_SUM = "largest_triple_sum"
    LONGEST_SEQ_LENGTH = "longest_seq_length"
    LARGEST_CONTIGUOUS_SUM = "largest_contiguous_sum"
    MAX = "max"
    CARDINALITY = "cardinality"

def load_regression_results() -> pd.DataFrame:
    part_1_results = pd.read_csv("regression-no-entropy.csv")
    part_2_results = pd.read_csv("regression-entropy.csv")
    part_2_results = part_2_results.drop(
        ["train_label_entropy", "valid_label_entropy", "test_label_entropy"],
        axis=1,
    )

    return pd.concat([part_1_results, part_2_results])

def load_classification_results() -> pd.DataFrame:
    return None

def compute_regression_task_results(results: pd.DataFrame, task: Task) -> pd.DataFrame:
    mean_results = results[results["label"] == task.value].groupby("max_set_size", as_index=False).mean()
    mean_results = mean_results.rename(
        columns={col: f"{col}_mean" for col in mean_results.columns}
    )
    std_results = results[results["label"] == task.value].groupby("max_set_size").std()
    std_results = std_results.rename(
        columns={col: f"{col}_std" for col in std_results.columns}
    )
    return mean_results

def compute_regression_model_results(results: pd.DataFrame, model_type: ModelType) -> pd.DataFrame:
    task_results = []
    for task in Task:
        tmp_results = compute_regression_task_results(
            results[results["type"] == model_type.value],
            task
        )
        tmp_columns = tmp_results.columns.copy()
        tmp_results["label"] = task.value
        task_results.append(tmp_results[["label", *tmp_columns]])
    return pd.concat(task_results)

def compute_classification_results_table(results: pd.DataFrame) -> pd.DataFrame:
    return results

def compute_regression_results_table(results: pd.DataFrame) -> pd.DataFrame:
    filtered_results = results[
        [
            "type",
            "n_params",
            "max_set_size",
            "label",
            "test_label_mean",
            "test_label_std",
            "epochs",
            "random_seed",
            "test_mse",
            "test_mae",
            "test_med_ae",
            "test_max_error",
            "test_r2",
        ]
    ]
    model_results = []
    for model_type in ModelType:
        tmp_results = compute_regression_model_results(filtered_results, model_type)
        tmp_columns = tmp_results.columns.copy()
        tmp_results["type"] = model_type.value
        model_results.append(tmp_results[["type", *tmp_columns]])
    return pd.concat(model_results)

def compute_results_table(kind: TaskType) -> None:
    if kind == TaskType.REGRESSION:
        results = load_regression_results()
        results = compute_regression_results_table(results)
        results = results[
            [
                "label",
                "type",
                "max_set_size_mean",
                "test_mae_mean",
                "test_max_error_mean",
                "test_r2_mean",
            ]
        ]
        results = results.sort_values(by=["max_set_size_mean", "label"], axis="index")
        results.to_csv(OutputFiles.regression_no_average, index=None)

        overall_averaged_results = results.groupby(["label", "type"]).mean()
        overall_averaged_results.drop(["max_set_size_mean"], axis=1, inplace=True)
        overall_averaged_results.to_csv(OutputFiles.regession_set_size_average)

        display(results)
        display(overall_averaged_results)
    elif kind == TaskType.CLASSIFICATION:
        results = load_classification_results()
        results = compute_classification_results_table(results)
    else:
        raise ValueError(f"Unknown task type: {kind}")

compute_results_table(TaskType.REGRESSION)